На основании базы знаний с ПРАВИЛАМИ СТРАХОВАНИЯ ОТВЕТСТВЕННОСТИ АЭРОПОРТОВ И АВИАЦИОННЫХ ТОВАРОПРОИЗВОДИТЕЛЕЙ создайте нейро-консультанта, который бы отвечал на вопросы клиентов по информации, содержащейся в представленном документе. Напишите промпт в system и user самостоятельно. Добейтесь того, чтобы нейро-консультант в своих ответах не ссылался на документы, переданные ему для ответа.

Документ с базой знаний можно найти тут: https://docs.google.com/document/d/11MU3SnVbwL_rM-5fIC14Lc3XnbAV4rY1Zd_kpcMuH4Y

In [ ]:
!pip install  -q langchain==0.1.4 tiktoken==0.5.2 faiss-cpu==1.7.4 sentence_transformers mistralai

ERROR: Could not install packages due to an OSError: Could not find a suitable TLS CA certificate bundle, invalid path: false



In [ ]:
import requests
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken
import matplotlib.pyplot as plt
import getpass
import os
from langchain.vectorstores import FAISS
import re
import getpass
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter, CharacterTextSplitter
from langchain.docstore.document import Document

c:\Users\sasakinme\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# запустите эту ячейку, если НЕ используете секретный ключ в колабе
import os
from mistralai import Mistral
import getpass

api_key = os.environ["MISTRAL_API_KEY"] 

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = Mistral(api_key)

In [ ]:
# запустите эту ячейку, если используете секретный ключ в колабе
import os
from mistralai import Mistral
from google.colab import userdata
api_key = userdata.get("MISTRAL_API_KEY")
os.environ["MISTRAL_API_KEY"] = api_key

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = Mistral(api_key)

In [17]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [18]:
# База знаний, которая будет подаваться в langChain
data_from_url= load_document_text('https://docs.google.com/document/d/11MU3SnVbwL_rM-5fIC14Lc3XnbAV4rY1Zd_kpcMuH4Y')

c:\Users\sasakinme\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'docs.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\sasakinme\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'doc-10-7c-docstext.googleusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [19]:
system = ''' Ты консультант.
Обеспечить пользователей точной и актуальной информацией по вопросам, связанным со страхованием гражданской ответственности в области авиации.

Твои задачи:

Объяснять основные концепции и термины, связанные со страхованием ответственности в авиационной сфере.
Предоставлять информацию о правах и обязанностях сторон, участвующих в страховании.
Ответить на вопросы о видах авиационных товаров и услуг, а также о деятельности аэропортов.
Помогать пользователям понимать, как страхование ответственности применяется в контексте авиационной отрасли.
Ключевые темы:

Страхование гражданской ответственности перед третьими лицами.
Определения и характеристики авиационных товаров и услуг.
Описание функций и задач аэропортов и аэропортовой деятельности.
Правила и условия, касающиеся страхования в авиационной сфере.
Формат общения:

Ответы должны быть четкими, лаконичными и информативными.
Использовать примеры для лучшего понимания сложных понятий.
Поддерживать вежливый и профессиональный стиль общения.
Ограничения:

Не предоставлять юридические консультации или рекомендации.
Избегать обсуждения вопросов, не относящихся к теме страхования ответственности в авиации. 
'''

In [20]:
system

' Ты консультант.\nОбеспечить пользователей точной и актуальной информацией по вопросам, связанным со страхованием гражданской ответственности в области авиации.\n\nТвои задачи:\n\nОбъяснять основные концепции и термины, связанные со страхованием ответственности в авиационной сфере.\nПредоставлять информацию о правах и обязанностях сторон, участвующих в страховании.\nОтветить на вопросы о видах авиационных товаров и услуг, а также о деятельности аэропортов.\nПомогать пользователям понимать, как страхование ответственности применяется в контексте авиационной отрасли.\nКлючевые темы:\n\nСтрахование гражданской ответственности перед третьими лицами.\nОпределения и характеристики авиационных товаров и услуг.\nОписание функций и задач аэропортов и аэропортовой деятельности.\nПравила и условия, касающиеся страхования в авиационной сфере.\nФормат общения:\n\nОтветы должны быть четкими, лаконичными и информативными.\nИспользовать примеры для лучшего понимания сложных понятий.\nПоддерживать веж

In [21]:
data_from_url

'\ufeffПРАВИЛА СТРАХОВАНИЯ ОТВЕТСТВЕННОСТИ АЭРОПОРТОВ И АВИАЦИОННЫХ ТОВАРОПРОИЗВОДИТЕЛЕЙ \r\n\r\n\r\n ОБЩИЕ ПОЛОЖЕНИЯ \r\n1.1. В соответствии с законодательством Российской Федерации и на основании настоящих Правил страховая организация АО «АльфаСтрахование» (далее по тексту – Страховщик) заключает договоры страхования гражданской ответственности перед третьими лицами владельцев и эксплуатантов аэропортов, поставщиков авиационных товаров и услуг, а также органов управления воздушным движением (далее по тексту - Страхователи). \r\n1.2. Основные понятия, конкретизированные определениями, изложенными ниже, трактуются в рамках настоящих Правил только согласно данным определениям: \r\n1.2.1. Авиационные товары и услуги - изделия авиационно-космической техники: самолеты, вертолеты, летательные аппараты специального назначения, планеры, автожиры, дельтапланы, ракеты космического назначения, космические аппараты, двигатели, агрегаты, оборудование, приборы и другие комплектующие изделия (КИ), к

In [22]:
source_chunks=[]
splitter = RecursiveCharacterTextSplitter(  )       # заполните нужными значениями, проведите эксперименты
for chunk in splitter.split_text(data_from_url):
    source_chunks.append(Document(page_content=chunk, metadata={}))
print("Общее количество чанков: ", len(source_chunks))

Общее количество чанков:  27


In [32]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# Функция для создания эмбеддингов из массива документов
def create_embeddings(documents):
    page_contents = [doc.page_content for doc in documents]
    return model.encode(page_contents)

# Создание эмбеддингов
embeddings = create_embeddings(source_chunks)

# Создадим индексную базу из разделенных фрагментов текста
import numpy as np
import faiss
# Преобразование эмбеддингов в массив numpy
embeddings = np.array(embeddings).astype('float32')

# Создание индекса FAISS
dimension = embeddings.shape[1]  # Размерность векторов
db = faiss.IndexFlatL2(dimension)  # Использование L2 метрики
# Добавление векторов в индекс
db.add(embeddings)

c:\Users\sasakinme\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\sasakinme\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\sasakinme\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-

In [48]:
# Функция, которая позволяет выводить ответ модели в удобочитаемом виде
def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return " ".join(lines)

def answer_index(system, topic, query_vector, fragments, search_index, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    distances, indices =  search_index.search(query_vector, k=4)
    #indices_list = indices[0].tolist()
    # Вывод ближайших документов
    docs = [fragments[i] for i in indices[0]]
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос. Документ с информацией для ответа: {message_content}\n\nВопрос пользователя: \n{topic}"}       # заполните нужными значениями, проведите эксперименты
    ]

    if verbose: print('\n ===========================================: ')
    client = Mistral(api_key)
    completion = client.chat.complete(
        model="mistral-small-latest",                               # заполните нужным значением
        messages=messages,
        temperature=0                           # заполните нужными значениями, проведите эксперименты
    )

    answer = insert_newlines(completion.choices[0].message.content, 50)
    return answer

In [31]:
source_chunks

[Document(metadata={}, page_content='\ufeffПРАВИЛА СТРАХОВАНИЯ ОТВЕТСТВЕННОСТИ АЭРОПОРТОВ И АВИАЦИОННЫХ ТОВАРОПРОИЗВОДИТЕЛЕЙ \r\n\r\n\r\n ОБЩИЕ ПОЛОЖЕНИЯ \r\n1.1. В соответствии с законодательством Российской Федерации и на основании настоящих Правил страховая организация АО «АльфаСтрахование» (далее по тексту – Страховщик) заключает договоры страхования гражданской ответственности перед третьими лицами владельцев и эксплуатантов аэропортов, поставщиков авиационных товаров и услуг, а также органов управления воздушным движением (далее по тексту - Страхователи). \r\n1.2. Основные понятия, конкретизированные определениями, изложенными ниже, трактуются в рамках настоящих Правил только согласно данным определениям: \r\n1.2.1. Авиационные товары и услуги - изделия авиационно-космической техники: самолеты, вертолеты, летательные аппараты специального назначения, планеры, автожиры, дельтапланы, ракеты космического назначения, космические аппараты, двигатели, агрегаты, оборудование, приборы и 

Проверьте работу консультанта на самостоятельно сформулированных вопросах.

In [49]:
topic= "Какие документы необходимы для оформления договора страхования гражданской ответственности?"                                      # Напишите сюда вопрос нейро-консультанту
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')

ans = answer_index(system, topic, query_vector, source_chunks, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================11.2. Существенными изменениями в страховом риске признаются такие изменения в объекте страхования или связанные с объектом страхования, которые имеют значение для определения вероятности наступления страхового случая и размера возможных убытков от его наступления, и которые могли бы – если бы они существовали в момент заключения договора страхования – повлиять на решение Страховщика о заключении договора страхования или на конкретные условия договора страхования. 11.3. В случаях, если Страхователь сомневается, являются ли произошедшие в страховом риске изменения существенными, он обязан уведомить Страховщика об этих изменениях. 
11.4. В общем случае существенными являются изменения в сведениях, предоставленных Страхователем при заключении договора страхования и/или указанных в договоре страхования. 
11.5. Страховщик, уведомленный об обстоятельствах, влекущих увеличение страхов

' Для оформления договора страхования гражданской  ответственности в авиационной сфере необходимо  предоставить следующие документы: 1. **Сведения о  Страхователе и Застрахованных лицах**: -  Наименование, адрес регистрации, ИНН. - Название  и адрес аэропорта, в котором ведется деятельность  Страхователя (если применимо). 2. **Сертификаты и  лицензии**: - Сертификаты, на основании которых  осуществляется авиационная деятельность. 3.  **Срок страхования**: - Указание периода, на  который заключается договор страхования. 4.  **Лимиты ответственности**: - Указание  максимальных сумм ответственности, которые будут  покрыты страховкой. 5. **Характер осуществляемого  бизнеса**: - Описание вида деятельности  Страхователя, включая авиационный и неавиационный  бизнес. 6. **Месторасположение и оборудование**:  - Информация о месторасположении помещений и  оборудования. 7. **Сотрудники и транспортные  средства**: - Количество сотрудников, работающих  в аэропорту. - Количество и тип эксплуатируемы

In [50]:
topic= "Какова процедура подачи претензии в случае наступления страхового случая?"                                      # Напишите сюда вопрос нейро-консультанту
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')

ans = answer_index(system, topic, query_vector, source_chunks, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================10.5. Страховщик по просьбе Страхователя, для предоставления заинтересованным лицам, вправе подтвердить факт заключения договора страхования путем выпуска отдельного подтверждения (сертификата). Такое подтверждение (сертификат) может содержать выписку из условий договора страхования, при этом условия договора страхования всегда превалируют над условиями такого подтверждения (сертификата). 10.6. Договор страхования заключается на любой согласованный сторонами срок, который устанавливается договором страхования. 
10.7. При заключении договора страхования на срок менее одного года страховая премия исчисляется в соответствии со следующей Таблицей, если договором страхования не предусмотрено иное: Срок действия договора в месяцах 1 2 3 4 5 6 7 8 9 10 11 Процент от общего годового размера страховой премии 20 30 40 50 60 70 75 80 85 90 95 
10.8. Договор страхования, если в нем не пред

' Процедура подачи претензии в случае наступления  страхового случая в контексте страхования  гражданской ответственности в авиационной сфере  обычно включает несколько ключевых шагов. Вот  основные этапы, которые могут быть предусмотрены  в договоре страхования и правилах страхования: 1.  **Уведомление Страховщика**: - **Срочность**:  Страхователь обязан незамедлительно уведомить  Страховщика о наступлении страхового случая. Это  может быть сделано по телефону, электронной почте  или другим способом, указанным в договоре  страхования. - **Документы**: При уведомлении  необходимо предоставить первичную информацию о  страховом случае, включая дату и время  происшествия, описание события и предварительную  оценку ущерба. 2. **Сбор и предоставление  документов**: - **Документы, подтверждающие  страховой случай**: Страхователь должен собрать и  предоставить все необходимые документы,  подтверждающие факт страхового случая. Это могут  быть полицейские протоколы, медицинские справки,  отчеты

In [51]:
topic= "Какие права и обязанности у меня как страхователя в рамках договора страхования?"                                      # Напишите сюда вопрос нейро-консультанту
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')

ans = answer_index(system, topic, query_vector, source_chunks, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================11.2. Существенными изменениями в страховом риске признаются такие изменения в объекте страхования или связанные с объектом страхования, которые имеют значение для определения вероятности наступления страхового случая и размера возможных убытков от его наступления, и которые могли бы – если бы они существовали в момент заключения договора страхования – повлиять на решение Страховщика о заключении договора страхования или на конкретные условия договора страхования. 11.3. В случаях, если Страхователь сомневается, являются ли произошедшие в страховом риске изменения существенными, он обязан уведомить Страховщика об этих изменениях. 
11.4. В общем случае существенными являются изменения в сведениях, предоставленных Страхователем при заключении договора страхования и/или указанных в договоре страхования. 
11.5. Страховщик, уведомленный об обстоятельствах, влекущих увеличение страхов

' В рамках договора страхования у вас, как  страхователя, есть определенные права и  обязанности, которые регулируются  законодательством Российской Федерации, договором  страхования и правилами страхования. Вот основные  из них: ### Права страхователя: 1. **Право на  получение страховой выплаты**: В случае  наступления страхового случая вы имеете право на  получение страховой выплаты в порядке, в сроки и  в размерах, предусмотренных договором страхования  и правилами страхования. 2. **Право на  уведомление о страховом случае**: Вы имеете право  уведомить страховщика о страховом случае и  предоставить все необходимые документы для  подтверждения факта, причины и размера убытка. 3.  **Право на получение сертификата**: По вашей  просьбе страховщик может выдать сертификат,  подтверждающий факт заключения договора  страхования. 4. **Право на досрочное прекращение  договора**: Вы можете потребовать досрочного  прекращения договора страхования, если это  предусмотрено договором и законодател

In [52]:
topic= "Каковы условия, при которых страховая компания может отказать в выплате страхового возмещения?"                                      # Напишите сюда вопрос нейро-консультанту
query_vector = model.encode([topic])
query_vector = np.array(query_vector).astype('float32')

ans = answer_index(system, topic, query_vector, source_chunks, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================11.2. Существенными изменениями в страховом риске признаются такие изменения в объекте страхования или связанные с объектом страхования, которые имеют значение для определения вероятности наступления страхового случая и размера возможных убытков от его наступления, и которые могли бы – если бы они существовали в момент заключения договора страхования – повлиять на решение Страховщика о заключении договора страхования или на конкретные условия договора страхования. 11.3. В случаях, если Страхователь сомневается, являются ли произошедшие в страховом риске изменения существенными, он обязан уведомить Страховщика об этих изменениях. 
11.4. В общем случае существенными являются изменения в сведениях, предоставленных Страхователем при заключении договора страхования и/или указанных в договоре страхования. 
11.5. Страховщик, уведомленный об обстоятельствах, влекущих увеличение страхов

' Страховая компания может отказать в выплате  страхового возмещения в нескольких случаях,  которые предусмотрены правилами страхования. Вот  основные условия, при которых это может  произойти: 1. **Непредставление необходимых  документов**: Если Страхователь или  Выгодоприобретатель не предоставят документы,  подтверждающие факт, причины и размер убытка,  требования о возмещении вреда, предъявленные  потерпевшими лицами, или решения судебных органов  о возмещении причиненного вреда (если дело  рассматривалось в судебном порядке), Страховщик  имеет право отказать в выплате возмещения в части  убытка, не подтвержденного документами (п. 12.2).  2. **Неуведомление о страховом случае**: Если  Страхователь не уведомит Страховщика о  происшествии, которое может послужить поводом для  предъявления претензии о возмещении причиненного  вреда, в течение суток с момента его обнаружения,  Страховщик может отказать в выплате страхового  возмещения (п. 13.5.8.2). 3. **Несоблюдение  условий договора*